In [19]:
! pip install gspread oauth2client pandas openai python-dotenv

In [20]:
from dotenv import load_dotenv
load_dotenv()

True

In [40]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
from datetime import datetime, timedelta
import openai
import json
import os

In [22]:
SHEET_NAME = "Fridge Inventory" 
SCOPE = [
    "https://spreadsheets.google.com/feeds",
    "https://www.googleapis.com/auth/spreadsheets",
    "https://www.googleapis.com/auth/drive"
]
creds = ServiceAccountCredentials.from_json_keyfile_name("google_creds.json", SCOPE)
client = gspread.authorize(creds)
sheet = client.open(SHEET_NAME).sheet1

In [23]:
# Fetch all rows from the sheet and load into pandas dataframe
records = sheet.get_all_records()
df = pd.DataFrame(records)

# Convert 'Expiry Date' and 'Purchase Date' to datetime objects
df["Expiry Date"] = pd.to_datetime(df["Expiry Date"], errors='coerce')
df["Purchase Date"] = pd.to_datetime(df["Purchase Date"], errors='coerce')

In [38]:
# Calculate today's date and threshold for expiry (2 days from now)
today = datetime.today()
threshold = today + timedelta(days=2)

# Keep only rows with quantity info and items expiring soon
filtered = df[
    (df["Expiry Date"] <= threshold) &
    (df["Remaining Quantity"].notna()) &
    (df["Remaining Quantity"] != "") &
    (df["Remaining Quantity"].astype(str).str.strip() != "0")
]

In [39]:
if filtered.reset_index(drop=True).empty:
    print("✅ No items are expiring in the next 2 days. You're good!")
else:
    print(filtered.reset_index(drop=True)[["Item Name", "Expiry Date", "Remaining Quantity"]])

          Item Name Expiry Date Remaining Quantity
0              Milk  2025-06-28                 1L
1          Broccoli  2025-06-27             1 head
2           Spinach  2025-06-27            1 bunch
3    Chicken Breast  2025-06-26              2 pcs
4       Cooked Rice  2025-06-26             1 bowl
5       Bell Pepper  2025-06-28                  1
6    Cooked Lentils  2025-06-26             1 bowl
7       Green Beans  2025-06-28               150g
8      Cooked Pasta  2025-06-26             1 bowl
9   Fresh Coriander  2025-06-28            1 bunch
10         Sausages  2025-06-27              4 pcs
11            Bacon  2025-06-27               200g
12        Mushrooms  2025-06-28               200g
13         Cucumber  2025-06-28                  2
14   Cooked Chicken  2025-06-26             1 bowl
15     Apple Slices  2025-06-27        1 container


In [42]:
# Prepare a list of unique expiring items to include in prompt
ingredient_list = [
    {
        "item": row["Item Name"],
        "quantity": row["Remaining Quantity"]
    }
    for _, row in filtered.iterrows()
]
ingredient_list

[{'item': 'Milk', 'quantity': '1L'},
 {'item': 'Broccoli', 'quantity': '1 head'},
 {'item': 'Spinach', 'quantity': '1 bunch'},
 {'item': 'Chicken Breast', 'quantity': '2 pcs'},
 {'item': 'Cooked Rice', 'quantity': '1 bowl'},
 {'item': 'Bell Pepper', 'quantity': 1},
 {'item': 'Cooked Lentils', 'quantity': '1 bowl'},
 {'item': 'Green Beans', 'quantity': '150g'},
 {'item': 'Cooked Pasta', 'quantity': '1 bowl'},
 {'item': 'Fresh Coriander', 'quantity': '1 bunch'},
 {'item': 'Sausages', 'quantity': '4 pcs'},
 {'item': 'Bacon', 'quantity': '200g'},
 {'item': 'Mushrooms', 'quantity': '200g'},
 {'item': 'Cucumber', 'quantity': 2},
 {'item': 'Cooked Chicken', 'quantity': '1 bowl'},
 {'item': 'Apple Slices', 'quantity': '1 container'}]

In [ ]:
if food_preference not in ["veg", "non-veg"]:
    food_preference = "veg"  # Default to vegetarian